In [1]:
import json

data_path = "train1.json"
with open(data_path, 'r') as f:
    data = json.load(f)

In [9]:
data['topic_list']

2

In [11]:
data['general_query_list']

[{'query': 'Summarize the whole meeting.',
  'answer': 'This was the kick-off meeting for the project. First of all, Project Manager led each group member to know each other and introduced the project which was aiming to design remote control. Next, they discussed their favourite animal characteristics. Lastly, Project Manager mentioned how they worked on each part individually.'}]

In [12]:
data['specific_query_list']

[{'query': "Summarize the groupmates' self-introduction and the project introduction.",
  'answer': 'There were four people in the project team and each one introduced to each other on the team role. Project Manager introduced the project was about designing a remote control. After that, Project Manager explained the work division for each person and how they would present in the coming meetings.',
  'relevant_text_span': [['0', '20']]},
 {'query': 'Summarize the job role for each groupmate.',
  'answer': 'The group was greeting each other at the first meeting. Laura was the Project Manager. David was Industrial Designer and Andrew was Marketing expert. And User Interface was named Craig.',
  'relevant_text_span': [['0', '9']]},
 {'query': 'What did the group discuss about the email they received on the project announcement?',
  'answer': 'Group mates all should have received an email introducing what was this project about and there would be three different stages to the design. The p

In [16]:
data['meeting_transcripts'][:5]

[{'speaker': 'Project Manager',
  'content': "Okay Right {vocalsound} Um well this is the kick-off meeting for our our project . Um {vocalsound} and um this is just what we're gonna be doing over the next twenty five minutes . Um so first of all , just to kind of make sure that we all know each other ,"},
 {'speaker': 'Marketing', 'content': 'Mm-hmm .'},
 {'speaker': 'Project Manager',
  'content': "I'm Laura and I'm the project manager . {vocalsound} Do you want to introduce yourself again ?"},
 {'speaker': 'Marketing', 'content': 'Great .'},
 {'speaker': 'Industrial Designer',
  'content': "Hi , I'm David and I'm supposed to be an industrial designer ."}]

In [28]:
def clean_data(text):
    text = text.replace('{vocalsound}', '')
    text = text.replace('{disfmarker} ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{pause} ', '')
    text = text.replace('{nonvocalsound} ', '')
    text = text.replace('{gap} ', '')
    return text

def extract_text_from_json(json_data):
    data = []
    for turn, meeting in enumerate(json_data['meeting_transcripts']):
        if meeting:
            data.append({
                'turn': turn,
                'speaker': meeting['speaker'],
                'content': clean_data(meeting['content']),
            })
    return data

In [29]:
temp = extract_text_from_json(data)
with open("data.jsonl", "w") as f:
    for example in temp:
        f.write(f"{json.dumps(example)}\n")

In [30]:
import datasets
import sentence_transformers

def get_embeddings(batch, model):
    embeddings = model.encode(batch["text"])
    return {"embeddings": embeddings}

model = sentence_transformers.SentenceTransformer(
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",
)

dataset = datasets.load_dataset("json", data_files="data.jsonl", split="train")
dataset = dataset.map(get_embeddings, batched=True, batch_size=32, fn_kwargs={"model": model})
dataset = dataset.with_format(
    type="numpy", columns=["embeddings"], output_all_columns=True,
)
dataset.add_faiss_index("embeddings")
dataset.save_faiss_index("embeddings", "index.faiss")

ModuleNotFoundError: No module named 'datasets'

In [2]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import JSONLoader

In [14]:
loader = JSONLoader(
    file_path='data.json',
    jq_schema='.[].content',
)
document = loader.load()
print(f'documents:{len(document)}')

documents:287


In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
# remove empty documents
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

documents:257


In [21]:
llm = OpenAI(model_name="text-davinci-003", max_tokens=1500)

chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

chain.run(split_documents[:5])

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)